If Dockerfiles have not been modified, connect to the Jupyter server with ```http://localhost:8003/tree?token=generate-gist```  

This notebook describes a pipeline to evaluate the GIST descriptor of an image.  
It takes an input ```.jpg``` (resized to 512×256px) and outputs its GIST descriptor as a 3000×1 matrix saved as a ```.csv```.  
This uses [lear-gist-python](https://github.com/whitphx/lear-gist-python/tree/master), a Python implementation of [Oliva and Torralba (2001)]((https://people.csail.mit.edu/torralba/code/spatialenvelope)), and [FFTW](https://www.fftw.org/).  

Oliva, A., Torralba, A. Modeling the Shape of the Scene: A Holistic Representation of the Spatial Envelope. International Journal of Computer Vision 42, 145–175 (2001). https://doi.org/10.1023/A:1011139631724

In [20]:
target_dir = "data"
input_dir = "shunfu-panos"
output_dir = "shunfu-gists"

In [21]:
import gist
import numpy as np
import PIL.Image as Image

import csv
import os
from pathlib import Path

In [22]:
images_path = []
for dirpath, dirnames, filenames in os.walk(os.path.join(target_dir, input_dir)):
    images_path.extend(filenames)
    break

Path(os.path.join(target_dir, output_dir)).mkdir(parents=True, exist_ok=True)

for image_path in images_path:
    if image_path.endswith(".jpg"):
        image = None
        with Image.open(os.path.join(target_dir, input_dir, image_path)) as im:
            image = np.asarray(im.resize((512, 256)))
        if image is not None:
            im_gist = gist.extract(image, nblocks=5, orientations_per_scale=(8, 8, 8, 8, 8)).tolist()
            with open(os.path.join(target_dir, output_dir, image_path.removesuffix(".jpg") + ".csv"), 'w') as fp:
                csv_writer = csv.writer(fp)
                for i in im_gist:
                    csv_writer.writerow([i])